In [29]:
from pyneuroml import pynml
import urllib.request, json 
import requests
import os
from neuroml import *
from neuroml.utils import component_factory, validate_neuroml2
from pyneuroml import pynml
from pyneuroml.lems import LEMSSimulation
import neuroml.writers as writers
import random
from pyneuroml.pynml import read_lems_file, read_neuroml2_file, write_lems_file, write_neuroml2_file


In [16]:
'''https://pyneuroml.readthedocs.io/en/development/pyneuroml.io.html
pyneuroml.io.read_lems_file
pyneuroml.io.read_neuroml2_file
pyneuroml.io.write_lems_file
pyneuroml.io.write_neuroml2_file'''

# use the above functions to read LEMS and NeuroML files
cell_data_dir = "../NMC_Model/NMC.NeuronML2/"
network_data_dir = "../NMC_Model/"  # contains JSONs for network and connectivity data

files = os.listdir(cell_data_dir)
cell_files = [f for f in files if f.endswith("cell.nml")]  # .cell.nml -- contains arborization location details
net_files = [f for f in files if f.endswith("net.nml")]  # .net.nml -- contains location (0,0,0) in x-y-z space
current_files = [f for f in files if (f.endswith(".nml") and 'current' in f)]  # .current_clamp.nml -- contains current clamp details
print(cell_files[:5])
print(net_files[:5])
print(current_files[:5])

# validate the NeuroML files
'''for file in cell_files:
    print("Validating: " + file)
    validate_neuroml2(cell_data_dir + file)
for file in net_files:
    validate_neuroml2(cell_data_dir + file)
for file in current_files:
    validate_neuroml2(cell_data_dir + file)
'''

['bAC217_L23_BP_2004171a4c_0_0.cell.nml', 'bAC217_L23_BP_2f6bdee09f_0_0.cell.nml', 'bAC217_L23_BP_4652c9a7e1_0_0.cell.nml', 'bAC217_L23_BP_472b717f1f_0_0.cell.nml', 'bAC217_L23_BP_ca85cefc69_0_0.cell.nml']
['bAC217_L23_BP_2004171a4c.net.nml', 'bAC217_L23_BP_2004171a4c.stepcurrent3.net.nml', 'bAC217_L23_BP_2f6bdee09f.net.nml', 'bAC217_L23_BP_2f6bdee09f.stepcurrent3.net.nml', 'bAC217_L23_BP_4652c9a7e1.net.nml']
['bAC217_L23_BP_2004171a4c.stepcurrent3.net.nml', 'bAC217_L23_BP_2f6bdee09f.stepcurrent3.net.nml', 'bAC217_L23_BP_4652c9a7e1.stepcurrent3.net.nml', 'bAC217_L23_BP_472b717f1f.stepcurrent3.net.nml', 'bAC217_L23_BP_ca85cefc69.stepcurrent3.net.nml']


'for file in cell_files:\n    print("Validating: " + file)\n    validate_neuroml2(cell_data_dir + file)\nfor file in net_files:\n    validate_neuroml2(cell_data_dir + file)\nfor file in current_files:\n    validate_neuroml2(cell_data_dir + file)\n'

In [32]:
# read in the network and connection properties from jsons: circuit.json, layers.json
with open(network_data_dir + 'circuit.json') as f:
    circuit_data = json.load(f)
with open(network_data_dir + 'layers.json') as f:
    layers_data = json.load(f)
layers_data

{'L6': {'No. of neurons per electrical types': {'cSTUT': 69,
   'cNAC': 142,
   'cADpyr': 9055,
   'bNAC': 89,
   'bSTUT': 106,
   'cIR': 32,
   'dNAC': 34,
   'bAC': 109,
   'cAC': 233,
   'dSTUT': 19,
   'bIR': 192},
  'number of intralaminar pathways': 176,
  'total axonal volume': 6277623.67,
  'No. of possible pathways': 770,
  'number of inhibitory pathways': 249,
  'No. of neurons per morphological types': {'L6_MC': 336,
   'L6_ChC': 16,
   'L6_SBC': 67,
   'L6_NGC': 17,
   'L6_LBC': 463,
   'L6_BTC': 54,
   'L6_NBC': 198,
   'L6_BPC': 3174,
   'L6_IPC': 3476,
   'L6_TPC_L1': 1637,
   'L6_DBC': 31,
   'L6_TPC_L4': 1440,
   'L6_UTPC': 1735,
   'L6_BP': 7},
  'number of excitatory pathways': 239,
  'total axonal length': 121932187.71,
  'total dendritic volume': 22738596.63,
  'total dendritic length': 81513471.45,
  'number of viable pathways': 488,
  'number of interlaminar pathways': 312,
  'synapse density': 105211259.0},
 'L23': {'dendritic length': 17237998.52,
  'No. of neu

In [59]:
for layer in layers_data:
    layer_dict = layers_data[layer]
    e_type_counts = layer_dict['No. of neurons per electrical types']
    m_type_counts = layer_dict['No. of neurons per morphological types']
    #print(e_type_counts)
    #print(m_type_counts)

    for m_type in m_type_counts:
        m_type_cells = [f for f in cell_files if m_type in f]
        m_count = m_type_counts[m_type]

        # for the e-types that match this m-type, build a relative probability distribution of these e-types based on their counts
        me_type_probs = {}
        e_count_sum = 0

        for e_type in e_type_counts:
            me_type_matches = [f for f in m_type_cells if e_type in f]
            for match in me_type_matches:
                me_type_probs[match] = e_type_counts[e_type] / len(me_type_matches)
            if len(me_type_matches) > 0:
                e_count_sum += e_type_counts[e_type]
        
        # normalize the probabilities
        for e_type in me_type_probs:
            me_type_probs[e_type] /= e_count_sum
        #print(me_type_probs)

        # probabilities should sum to 1
        if abs(1 - sum(me_type_probs.values())) > 0.0001 and len(me_type_probs.keys()) > 0:
            print("Error: probabilities do not sum to 1")
            print(sum(me_type_probs.values()))
            print(me_type_probs)


        # sample from the me-type distribution to get the number of cells of each type
        for i_morp in range(m_count):
            m_type_cells = list(me_type_probs.keys())
            m_type_probs = list(me_type_probs.values())
            if len(m_type_cells) == 0 or len(m_type_probs) == 0:
                continue
            me_type_sample = random.choices(m_type_cells, m_type_probs)[0]
            print("\t", me_type_sample)

            # read in the cell file




	 ['bNAC219_L6_MC_df869132b9_0_0.cell.nml']
	 ['cIR216_L6_MC_dc1f55aab0_0_0.cell.nml']
	 ['bIR215_L6_MC_62cf39b45d_0_0.cell.nml']
	 ['cACint209_L6_MC_5597e4d536_0_0.cell.nml']
	 ['cACint209_L6_MC_5597e4d536_0_0.cell.nml']
	 ['cACint209_L6_MC_792eb41cce_0_0.cell.nml']
	 ['cNAC187_L6_MC_5597e4d536_0_0.cell.nml']
	 ['bNAC219_L6_MC_62cf39b45d_0_0.cell.nml']
	 ['bIR215_L6_MC_62cf39b45d_0_0.cell.nml']
	 ['bNAC219_L6_MC_6c575906a7_0_0.cell.nml']
	 ['bIR215_L6_MC_6c575906a7_0_0.cell.nml']
	 ['bSTUT213_L6_MC_8f46a0ad62_0_0.cell.nml']
	 ['bSTUT213_L6_MC_df869132b9_0_0.cell.nml']
	 ['cACint209_L6_MC_df869132b9_0_0.cell.nml']
	 ['bSTUT213_L6_MC_e8d04feaa6_0_0.cell.nml']
	 ['cACint209_L6_MC_62cf39b45d_0_0.cell.nml']
	 ['bIR215_L6_MC_6ed36072e7_0_0.cell.nml']
	 ['bNAC219_L6_MC_62cf39b45d_0_0.cell.nml']
	 ['bIR215_L6_MC_8f46a0ad62_0_0.cell.nml']
	 ['cNAC187_L6_MC_8f46a0ad62_0_0.cell.nml']
	 ['cNAC187_L6_MC_6c575906a7_0_0.cell.nml']
	 ['cIR216_L6_MC_a405be0ec4_0_0.cell.nml']
	 ['bSTUT213_L6_MC_df86913

In [ ]:

nml_doc = component_factory("NeuroMLDocument", id="NML_DB_Net")


### Create the network
net = nml_doc.add("Network", id="NML_DB_Net", validate=False)
net.type="networkWithTemperature" 
net.temperature="34.0degC"

### Add a synapse
syn0 = nml_doc.add(
    "ExpOneSynapse", id="syn0", gbase="65nS", erev="0mV", tau_decay="3ms"
)


## Create the first population
size_exc = 4
nml_doc.add("IncludeType", href=cell_file)

pop_exc = component_factory("Population", id="Exc", component=cell.id, size=size_exc, type="population")
# Set optional color property. Note: used later when generating plots
##pop0.add("Property", tag="color", value="0 0 .8")
net.add(pop_exc)

## Create the second population
size_inh = 4
nml_doc.add("IncludeType", href=detailed_cell_file)

pop_inh = component_factory("Population", id="Inh", component=detailed_cell.id, size=size_inh, type="population")
# Set optional color property. Note: used later when generating plots
##pop1.add("Property", tag="color", value="0 0 .8")
net.add(pop_inh)

## Create connections and inputs
random.seed(123)
prob_connection = 0.8

proj_count = 0

projection = Projection(
                    id="Proj_exc_inh",
                    presynaptic_population=pop_exc.id,
                    postsynaptic_population=pop_inh.id,
                    synapse=syn0.id,
                )

net.projections.append(projection)

for i in range(0, size_exc):
    for j in range(0, size_inh):
        if random.random() <= prob_connection: # probablistic connection...
            connection = ConnectionWD(
                id=proj_count,
                pre_cell_id="%s[%i]" % (pop_exc.id, i),
                post_cell_id="%s[%i]" % (pop_inh.id, j),
                weight=random.random(),
                delay='0ms'
            )
            projection.add(connection)
            proj_count += 1

for i in range(0, size_exc):
    # pulse generator as explicit stimulus
    pg = nml_doc.add(
        "PulseGenerator",
        id="pg_exc_%i" % i,
        delay="20ms",
        duration="260ms",
        amplitude="%f nA" % (0.03 + 0.01 * random.random()),
    )
    
    exp_input = net.add(
        "ExplicitInput", target="%s[%i]" % (pop_exc.id, i), input=pg.id
    )

for i in range(0, size_inh):
    # pulse generator as explicit stimulus
    pg = nml_doc.add(
        "PulseGenerator",
        id="pg_inh_%i"%i,
        delay="20ms",
        duration="260ms",
        amplitude="%f nA" % (0.02 + 0.02 * random.random()),
    )
    
    exp_input = net.add(
        "ExplicitInput", target="%s[%i]" % (pop_inh.id, i), input=pg.id
    )


print(nml_doc.summary())

nml_net_file = 'NML_DB_network.net.nml'
writers.NeuroMLWriter.write(nml_doc, nml_net_file)

print("Written network file to: " + nml_net_file)
pynml.validate_neuroml2(nml_net_file)